# Using generative AI with structured data for programmatic SEO

Prompt engineering is one of the hot topics these days, and it is constantly changing due to rapid changes in the guidelines, the underlying technologies' capabilities, as well as our understanding of them.

It is mostly known as making many attempts, changing, rewriting, until you find a prompt that works for the task at hand.

I would like to build on this and share a pattern of using building prompt **templates** that can be used in bulk, for the creation of product descriptions, email marketing messages, and any other type of content.

The key enabler in this pattern is the availability of strcutured data for many items, and for a consitent set of attributes.

product   | size | color | price
----------|------|-------|-------
Product A |10    |blue   | 150
Product B |12    |green  | 180
Product C |15    |red    | 125


A very simple prompt for a product description for one of those products can be something like this:


> Please create a product description for **Product A**
Mention its size, which is **10**
Also mention its color, which is **blue**
And talk about its price: **150**
The tone of voice should be professional

Now we have two important things to consider:
1. We would like to scale our prompting to encompass all our products.
2. The consistency of our data enables us to run the same process across many products.
3. It would be great if we can save the process that generated our product descriptions. This is in order to debug issues, reproduce them again, and eventually update them by tweaking the prompt or by adding new product attributes whenever we have updated data.


In [7]:
import pandas as pd
products = pd.DataFrame({
    'product': ['Product A', 'Product B', 'Product C'],
    'size': [10, 12, 15],
    'color': ['blue', 'green', 'red'],
    'price': [150, 180, 125]
})
products

,product,size,color,price
0,Product A,10,blue,150
1,Product B,12,green,180
2,Product C,15,red,125


In [41]:
prompt_template = """
Please create a product description for {product}.
Mention its size, which is {size}
Also mention its color, which is {color}
Talk about its price: {price}
The tone of voice should be professional
"""

for row in products.to_dict('records'):
    print(prompt_template.format_map(row))


Please create a product description for Product A.
Mention its size, which is 10
Also mention its color, which is blue
Talk about its price: 150
The tone of voice should be professional


Please create a product description for Product B.
Mention its size, which is 12
Also mention its color, which is green
Talk about its price: 180
The tone of voice should be professional


Please create a product description for Product C.
Mention its size, which is 15
Also mention its color, which is red
Talk about its price: 125
The tone of voice should be professional



Let's take a quick look at some of the adavantages of this approach:

## Reproducibility
Eventually we will most likely want to update those descriptions. We might have a new SEO strategy and want to focus on other aspects of our products. We might get new data, like user ratings for example, and might want to incorporate a new line in our prompt, and so on.

## Scaling
I think this is obvious. This approach clearly allows us to create descriptions on a large scale, provided we have consitent data, and that the items are similar (belonging to the same product category, or are of the same type).

## Debugging
If we see something wrong in our product descriptions, or some unwanted behavior, we can always go back and update our prompt template and clarify in a better way what we want.

## Minimize hallucinations
Trusting LLMs to create product descriptions for you is a very risky thing, because they are well-known for how easily they can invent "facts", and being "confident" about them.
With this approach, we are restricting the factual information to the data that we provide, and we are also utilizing the great language processing capabilities of LLMs.

Now that we have clarified the approach and its benefits, let's see how this might be used in a real-life situation.

## How to use LLMs for product descriptions on a large scale

The example I will go through is for NBA player statistics. We will connect to the NBA data API, obtain carerr data for each player (per season, and per metric), and use that to create profile pages for each player.

## Added value of using LLMs for product descriptions

What we are doing here is not a simple re-arrangement of what someone else wrote, and calling it our own. This would not add real value, and would not be an ethical thing to do. We are using publicly available data to uncover some insights, and with those facts create an interesting description. This description should make the reader better informed about the topic discussed.

### Insights

With the availability of structured data about our players, we can make some calculations, and uncover some interesting facts that might not be immediately visible from the raw data.


### Charts

Another added value of using raw data is that we can visualize for truly more insightful views into the players' careers.

How easy is it to understand [Michael Jordan's career stats](https://nbastats.pro/player/Michael_Jordan) by looking at this table?

![](michael_jordan_career_stats_table.png)

Maybe this is better:

![](michael_jordan_career_stats_chart.png)

Was it easy for you to spot the two seasons where he played a very low number of games, and that otherwise he played pretty much all games of all seasons?

Was it clear from the table, that after his third year, his total points per season kept going down throughout his career (even though the games he played was stable, with two exceptions)?

Of course those charts are not produced by the LLM. We will produce them with Plotly, a Python data visualization library, and of course we can use other libraries for that.

## Creating prompt templates

Just like with did with the toy example above, we will now create a prompt template for NBA players.

### Importing libraries

In [78]:
import pandas as pd
from openai import OpenAI
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players, teams
import plotly.express as px
pd.options.display.max_columns = None
import os
from IPython.display import display_markdown
def md(text):
    return display_markdown(text, raw=True)
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

## Getting static data about players and teams (ID's, full name, city, state, etc.)

In [4]:
players_static = players.get_players()

In [11]:
pd.DataFrame(players_static).to_csv('players_static.csv', index=False)

In [7]:
teams_static = teams.get_teams()

In [12]:
pd.DataFrame(teams_static).to_csv('teams_static.csv', index=False)

In [39]:
players_static = pd.read_csv('players_static.csv')
teams_static = pd.read_csv('teams_static.csv')

In [41]:
players_static.sample(10)

,id,full_name,first_name,last_name,is_active
3393,77825,George Pearcy,George,Pearcy,False
1849,1538,Cedric Henderson,Cedric,Henderson,False
4077,22,Rik Smits,Rik,Smits,False
2829,77517,Paul McCracken,Paul,McCracken,False
3022,600006,Earl Monroe,Earl,Monroe,False
4324,78340,Mel Thurston,Mel,Thurston,False
3460,2565,Zoran Planinic,Zoran,Planinic,False
1782,203914,Gary Harris,Gary,Harris,True
42,203128,Furkan Aldemir,Furkan,Aldemir,False
1094,76577,Terry Dischinger,Terry,Dischinger,False


In [42]:
teams_static.sample(10)

,id,full_name,abbreviation,nickname,city,state,year_founded
13,1610612750,Minnesota Timberwolves,MIN,Timberwolves,Minnesota,Minnesota,1989
15,1610612752,New York Knicks,NYK,Knicks,New York,New York,1946
27,1610612764,Washington Wizards,WAS,Wizards,Washington,District of Columbia,1961
21,1610612758,Sacramento Kings,SAC,Kings,Sacramento,California,1948
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970
19,1610612756,Phoenix Suns,PHX,Suns,Phoenix,Arizona,1968
14,1610612751,Brooklyn Nets,BKN,Nets,Brooklyn,New York,1976
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946


In [43]:
players_static[players_static['full_name'].isin(['LeBron James', 'Michael Jordan'])]

,id,full_name,first_name,last_name,is_active
2111,2544,LeBron James,LeBron,James,True
2291,893,Michael Jordan,Michael,Jordan,False


## Get player career stats using the player's ID

In [24]:
lebron_james = playercareerstats.PlayerCareerStats(player_id=2544)
michael_jordan = playercareerstats.PlayerCareerStats(player_id=893)

In [33]:
lebron_james.get_data_frames()[0].head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,2544,2003-04,00,1610612739,CLE,19.0,79,79,3120.0,622,1492,0.417,63,217,0.290,347,460,0.754,99,333,432,465,130,58,273,149,1654
1,2544,2004-05,00,1610612739,CLE,20.0,80,80,3388.0,795,1684,0.472,108,308,0.351,477,636,0.750,111,477,588,577,177,52,262,146,2175
2,2544,2005-06,00,1610612739,CLE,21.0,79,79,3361.0,875,1823,0.480,127,379,0.335,601,814,0.738,75,481,556,521,123,66,260,181,2478
3,2544,2006-07,00,1610612739,CLE,22.0,78,78,3190.0,772,1621,0.476,99,310,0.319,489,701,0.698,83,443,526,470,125,55,250,171,2132
4,2544,2007-08,00,1610612739,CLE,23.0,75,74,3027.0,794,1642,0.484,113,359,0.315,549,771,0.712,133,459,592,539,138,81,255,165,2250


In [34]:
michael_jordan.get_data_frames()[0].head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,893,1984-85,00,1610612741,CHI,22.0,82,82,3144.0,837,1625,0.515,9,52,0.173,630,746,0.845,167,367,534,481,196,69,291,285,2313
1,893,1985-86,00,1610612741,CHI,23.0,18,7,451.0,150,328,0.457,3,18,0.167,105,125,0.840,23,41,64,53,37,21,45,46,408
2,893,1986-87,00,1610612741,CHI,24.0,82,82,3281.0,1098,2279,0.482,12,66,0.182,833,972,0.857,166,264,430,377,236,125,272,237,3041
3,893,1987-88,00,1610612741,CHI,25.0,82,82,3311.0,1069,1998,0.535,7,53,0.132,723,860,0.841,139,310,449,485,259,131,252,270,2868
4,893,1988-89,00,1610612741,CHI,26.0,81,81,3255.0,966,1795,0.538,27,98,0.276,674,793,0.850,149,503,652,650,234,65,290,247,2633


## Create a prompt template and dynamically insert each player's data

In [80]:
responses = []
for player_id in [2544, 893, 203999, 76003]:
    player_stats_df = playercareerstats.PlayerCareerStats(player_id=player_id).get_data_frames()[0]
    player_stats_df = pd.merge(
        player_stats_df,
        players_static,
        left_on='PLAYER_ID',
        right_on='id',
        how='left')
    player_stats_df = pd.merge(
        player_stats_df,
        teams_static,
        left_on='TEAM_ID',
        right_on='id',
        how='left')
    display(player_stats_df)

    # player_id = int(player_id.replace('player_', '').replace('.csv', ''))
    # t0 = time.time()
    try:
        df = player_stats_df[player_stats_df['PLAYER_ID'].eq(player_id)]
        if (df['FGM'].sum() > 0) and  (df['FGA'].sum() > 0):
            goals_percent = df['FGM'].sum() / df['FGA'].sum()
        else:
            goals_percent = 'unknown'
        if (df['FTM'].sum() > 0) and (df['FTA'].sum() > 0):
            freethrows_percent = df['FTM'].sum() / df['FTA'].sum() > 0
        else:
            freethrows_percent = 'unknown'
        d = dict(
            name = df['full_name_x'].iloc[0],
            teams = df['full_name_y'].drop_duplicates().dropna().tolist(),
            cities = df['city'].drop_duplicates().dropna().tolist(),
            states = df['state'].drop_duplicates().dropna().tolist(),
            active = df['is_active'].iloc[-1],
            start_season = df['SEASON_ID'].iloc[0],
            end_season = df['SEASON_ID'].iloc[-1],
            start_age = df['PLAYER_AGE'].iloc[0],
            end_age = df['PLAYER_AGE'].iloc[-1],
            games_played = df['GP'].sum(),
            minutes_played = df['MIN'].sum(),
            goals_attempted = df['FGA'].sum(),
            goals_made = df['FGM'].sum(),
            goals_percent = goals_percent,
            freethrows_attempted = df['FTA'].sum(),
            freethrows_made = df['FTM'].sum(),
            freethrows_percent = freethrows_percent,
            rebounds = df['REB'].sum(),
            rebounds_def = df['DREB'].sum(),
            rebounds_off = df['OREB'].sum(),
            assists = df['AST'].sum(),
            steals = df['STL'].sum(),
            blocks = df['BLK'].sum(),
            points = df['PTS'].sum(),
    )
        prompt = [
            {"role": "system",
             "content": """
You are a smart, detail-oriencted, keen NBA Basketball player analyst.
Please write an introductory text for a profile page of this Basketball player.
Length: 500 - 800 words.
please stick to the stats provided.
Tone: should be interesting factual intriguing and inviting the user to dive
into the charts on the page to better get to know the player."""},
             {"role": "user",
              "content": f"""
Please write and article for the player using the following details:
{d}"""}]
        display(md(f"## {df['full_name_x'].iloc[0]}"))
        for k, v in prompt[0].items():
            print(k, v)
        print()
        for k, v in d.items():
            print(k, v)
    except Exception as e:
        print(str(e))
        continue
    completion = client.chat.completions.create(
      # model="gpt-4o",
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system",
         "content": """You are a smart, detail-oriencted, keen NBA Basketball player analyst. \
         Please write an introductory text for a profile page of this Basketball player.
         Length: 500 - 800 words.
         please stick to the stats provided.
         Tone: should be interesting factual intriguing and inviting the user to dive \
         into the charts on the page to better get to know the player.
         """},
        {"role": "user",
         "content": f"""Please write and article for the player using the following details:
         {d}
         """}
      ]
    )
    responses.append([player_id, completion])
    print()
    print(completion.dict()['choices'][0]['message']['content'])
    print()
    display(md('----'))

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,id_x,full_name_x,first_name,last_name,is_active,id_y,full_name_y,abbreviation,nickname,city,state,year_founded
0,2544,2003-04,00,1610612739,CLE,19.0,79,79,3120.0,622,1492,0.417,63,217,0.290,347,460,0.754,99,333,432,465,130,58,273,149,1654,2544,LeBron James,LeBron,James,True,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
1,2544,2004-05,00,1610612739,CLE,20.0,80,80,3388.0,795,1684,0.472,108,308,0.351,477,636,0.750,111,477,588,577,177,52,262,146,2175,2544,LeBron James,LeBron,James,True,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
2,2544,2005-06,00,1610612739,CLE,21.0,79,79,3361.0,875,1823,0.480,127,379,0.335,601,814,0.738,75,481,556,521,123,66,260,181,2478,2544,LeBron James,LeBron,James,True,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,2544,2006-07,00,1610612739,CLE,22.0,78,78,3190.0,772,1621,0.476,99,310,0.319,489,701,0.698,83,443,526,470,125,55,250,171,2132,2544,LeBron James,LeBron,James,True,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
4,2544,2007-08,00,1610612739,CLE,23.0,75,74,3027.0,794,1642,0.484,113,359,0.315,549,771,0.712,133,459,592,539,138,81,255,165,2250,2544,LeBron James,LeBron,James,True,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
5,2544,2008-09,00,1610612739,CLE,24.0,81,81,3054.0,789,1613,0.489,132,384,0.344,594,762,0.780,106,507,613,587,137,93,241,139,2304,2544,LeBron James,LeBron,James,True,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
6,2544,2009-10,00,1610612739,CLE,25.0,76,76,2966.0,768,1528,0.503,129,387,0.333,593,773,0.767,71,483,554,651,125,77,261,119,2258,2544,LeBron James,LeBron,James,True,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
7,2544,2010-11,00,1610612748,MIA,26.0,79,79,3063.0,758,1485,0.510,92,279,0.330,503,663,0.759,80,510,590,554,124,50,284,163,2111,2544,LeBron James,LeBron,James,True,1610612748,Miami Heat,MIA,Heat,Miami,Florida,1988
8,2544,2011-12,00,1610612748,MIA,27.0,62,62,2326.0,621,1169,0.531,54,149,0.362,387,502,0.771,94,398,492,387,115,50,213,96,1683,2544,LeBron James,LeBron,James,True,1610612748,Miami Heat,MIA,Heat,Miami,Florida,1988
9,2544,2012-13,00,1610612748,MIA,28.0,76,76,2877.0,765,1354,0.565,103,254,0.406,403,535,0.753,97,513,610,551,129,67,226,110,2036,2544,LeBron James,LeBron,James,True,1610612748,Miami Heat,MIA,Heat,Miami,Florida,1988


## LeBron James

None

role system
content 
You are a smart, detail-oriencted, keen NBA Basketball player analyst.
Please write an introductory text for a profile page of this Basketball player.
Length: 500 - 800 words.
please stick to the stats provided.
Tone: should be interesting factual intriguing and inviting the user to dive
into the charts on the page to better get to know the player.

name LeBron James
teams ['Cleveland Cavaliers', 'Miami Heat', 'Los Angeles Lakers']
cities ['Cleveland', 'Miami', 'Los Angeles']
states ['Ohio', 'Florida', 'California']
active True
start_season 2003-04
end_season 2023-24
start_age 19.0
end_age 39.0
games_played 1492
minutes_played 56596.0
goals_attempted 29313
goals_made 14837
goals_percent 0.5061576774809812
freethrows_attempted 11404
freethrows_made 8390
freethrows_percent True
rebounds 11185
rebounds_def 9458
rebounds_off 1727
assists 11009
steals 2275
blocks 1111
points 40474

LeBron James: Dominating the Court with Unmatched Brilliance

When it comes to pure baske

----

...

## Kareem Abdul-Jabbar

None

role system
content 
You are a smart, detail-oriencted, keen NBA Basketball player analyst.
Please write an introductory text for a profile page of this Basketball player.
Length: 500 - 800 words.
please stick to the stats provided.
Tone: should be interesting factual intriguing and inviting the user to dive
into the charts on the page to better get to know the player.

name Kareem Abdul-Jabbar
teams ['Milwaukee Bucks', 'Los Angeles Lakers']
cities ['Milwaukee', 'Los Angeles']
states ['Wisconsin', 'California']
active False
start_season 1969-70
end_season 1988-89
start_age 23.0
end_age 42.0
games_played 1560
minutes_played 57446.0
goals_attempted 28307
goals_made 15837
goals_percent 0.5594729218921115
freethrows_attempted 9304
freethrows_made 6712
freethrows_percent True
rebounds 17440
rebounds_def 9394.0
rebounds_off 2975.0
assists 5660
steals 1160.0
blocks 3189.0
points 38387

Introducing Kareem Abdul-Jabbar, a legendary NBA Basketball player who made a lasting impact on the game dur

----

None